### 1. Keras ensemble
* test data set에 대해 Keras ensemble 결과

In [ ]:
import glob
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

class_num = 6

test_files = glob.glob("5_dataset/test/*/*.jpg")
y_temp_test = []
test_temp_files = []

for _file in test_files:
    test_temp_files.append(_file)
    _label = _file.split("\\")
    y_temp_test.append(int(_label[1][4]))  
    
image_width = 331
image_height = 221
channels = 3
x_test = np.ndarray(shape = (len(test_temp_files), image_width, image_height, channels), dtype = np.float32)
y_test = np.ndarray(shape = (len(y_temp_test), class_num), dtype = np.float32)

In [ ]:
def one_hot_encoding(num):
    arr = [0, 0, 0, 0, 0, 0]
    for i in range(class_num):
        if (i==num):
            arr[i] = 1
        else:
            arr[i] = 0
    return arr

In [ ]:
i = 0
for _file in test_files:
    img = load_img(_file)
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((331, 221, 3))
    x = (x-128.0)/128.0
    x_test[i] = x
    y_test[i] = one_hot_encoding(y_temp_test[i])
    i += 1
print("test data converting complete")

In [ ]:
import glob
from keras.models import load_model
from tqdm import tqdm_notebook
fileList = glob.glob("6_ModelBackUp/*.h5")
testList = glob.glob("5_dataset/test/*/*.jpg")

models = list()
for file in tqdm_notebook(fileList):
    models.append(load_model(file))
    
num = 0
real_value = 0
real_value_cnt = 0
real_value_cnt_include_background = 0
for i in tqdm_notebook(range(len(testList))):
    
    classPredict = [0, 0, 0, 0, 0, 0]
    answer = 5
    maximum = 0
    maxIndex = 0
    
    
    for model in models:
        tempPredict = model.predict_classes(x_test[i:i+1, :], verbose = 0)
        classPredict[int(tempPredict)] += 1
    for j in range(6):
        if(classPredict[j] > maximum):
            overlap = 0
            maximum = classPredict[j]
            maxIndex = j
    print(num, end = " -> ")
    num += 1
    print(classPredict, end = " : ")
    answer = maxIndex
    print(answer)
    if(answer == real_value):
        real_value_cnt+=1
        real_value_cnt_include_background += 1
    elif(answer == 5):
        real_value_cnt_include_background += 1
    if(i==86 or i == 285 or i == 484 or i == 559 or i == 745 or i == 943):
        # 전체 개수, 5포함 개수, 5불포함 개수
        print(str(real_value) + " 판별 개수 : " + str(real_value_cnt) + ", 배경 포함 개수 : " + str(real_value_cnt_include_background))
        real_value +=1
        real_value_cnt = 0
        real_value_cnt_include_background = 0
        print("\n===========================================================================")
        print("\n")

In [ ]:
print("test 100개당 잘못된 검출 횟수")
print("car_horn 온전한 error 비율", end = " : ")
print(round(((86-64)/86)*100, 3))
print("car_horn 배경 제외 error 비율", end = " : ")
print(round(((86-78)/86)*100, 3))
print("=============================")
print("dog_bark 온전한 error 비율", end = " : ")
print(round(((285-86+1-152)/(285-86+1))*100, 3))
print("dog_bark 배경 제외 error 비율", end = " : ")
print(round(((285-86+1-170)/(285-86+1))*100, 3))
print("=============================")
print("drilling 온전한 error 비율", end = " : ")
print(round(((484 -285+1-167)/(484 -285+1))*100, 3))
print("drilling 배경 제외 error 비율", end = " : ")
print(round(((484 -285+1-186)/(484 -285+1))*100, 3))
print("=============================")
print("gun_shot 온전한 error 비율", end = " : ")
print(round(((559  -484+1-70)/(559  -484+1))*100, 3))
print("gun_shot 배경 제외 error 비율", end = " : ")
print(round(((559  -484+1-71)/(559  -484+1))*100, 3))
print("=============================")
print("siren 온전한 error 비율", end = " : ")
print(round(((745-559+1-137)/(745-559+1))*100, 3))
print("siren 배경 제외 error 비율", end = " : ")
print(round(((745-559+1-167)/(745-559+1))*100, 3))
print("=============================")
print("배경잡음 온전한 error 비율", end = " : ")
print(round(((943-745+1-156)/(943-745+1))*100, 3))
print("=============================")

* class0 : car_horn
* class1 : dog_bark
* class2 : drilling
* class3 : gun_shot
* class4 : siren

---

### 2. 유튜브에서 실험 데이터 수집
* wav 형식으로 포맷 변화 필요

In [ ]:
# url 저장 배열
Horn = [
    "https://www.youtube.com/watch?v=i9ADKqS5LBg",
    "https://www.youtube.com/watch?v=HsxeUeKfd90",
    "https://www.youtube.com/watch?v=K8Sm67RHB_s"
]

ambulance = [
    "https://www.youtube.com/watch?v=xCZEr2BchbU", #0:25~0:29
    "https://www.youtube.com/watch?v=haqqpOFTgII", #noise 많음
    "https://www.youtube.com/watch?v=-2YWkWhIVtQ" #noise 많음, 0:00~0:17까지
]

fireTruck = [
    "https://www.youtube.com/watch?v=xCZEr2BchbU", #0:15~0:18
    "https://www.youtube.com/watch?v=hxVD4fA2YxM",
    "https://www.youtube.com/watch?v=gXnRkrYyFNg&t=23s"
]

fireAlarm = [
    "https://www.youtube.com/watch?v=GbvOrpW3yo8",
    "https://www.youtube.com/watch?v=CGxd9FOCLfQ",
    "https://www.youtube.com/watch?v=9v2TOr2aWX0" # 5:52 ~ 6:09
]

# url 저장 배열의 배열
ClassName1 = [
    'fireTruck',
    'ambulance',
    'fireAlarm',
    'Horn'
]

ClassName2 = {
    'fireTruck':fireTruck,
    'ambulance':ambulance,
    'fireAlarm':fireAlarm,
    'Horn':Horn
}

In [ ]:
def checkNumber(string):
    if(string[0]=='1' or string[0]=='2' or string[0]=='3' or string[0]=='4' or string[0]=='5' or string[0]=='6' or string[0]=='7' or string[0]=='8' or string[0]=='9'):
        return int(string[0:])
    elif(string[1]=='1' or string[1]=='2' or string[1]=='3' or string[1]=='4' or string[1]=='5' or string[1]=='6' or string[1]=='7' or string[1]=='8' or string[1]=='9'):
        return int(string[1:])
    else:
        return int(string[2])

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pafy
import webbrowser
import os
import glob

def geturl(val):
    s = "i.ytimg.com/vi/"
    return val[val.find(s)+len(s):val.find(s)+len(s)+11]

#for i in range(len(ClassName1)):
for i in range(1):
    keyword = ClassName1[i]
    directoryName = "Tempsound/"+keyword
    if not(os.path.isdir(directoryName)):
        os.makedirs(os.path.join(directoryName))
    fileList = glob.glob(directoryName+"/*.wav")
    # 10의 자리개의 파일이 되면 9보다 10을 큰수로 인식하는 함수 필요
    curNum = checkNumber(fileList[-1][-7:-4])+1
    print("========================================")
    print("Last save file number is " + str(curNum-1))
    for j in range(len(ClassName2[keyword])):
        try:            
            originUrl = ClassName2[keyword][j]
            r = requests.get(originUrl+'"topic"').text
            url = geturl(r)
            video = pafy.new(url)
            print(video)
            bestaudio = video.getbestaudio()
            print(bestaudio)
            url = bestaudio.url
            SaveName = keyword
            bestaudio.download(directoryName+"/"+keyword+str(j+curNum)+".wav")
            print("---------------------------")  
        except:
            print("error")
            break

---

### 3. 실제 데이터에 대해 Ensemble 기법 적용 및 가중치 조절 실험